# Transfer Learning 개념 이해:

사전 학습된 모델(Pre-trained Model)을 활용하여, 기존에 대규모 데이터셋으로 학습된 모델의 가중치를 재사용하고, 이를 우리의 특정 문제에 맞게 파인튜닝(Fine-tuning)하는 개념을 이해합니다.

# Pre-trained 모델 활용:

torchvision에서 제공하는 Pre-trained 모델(예: ResNet, VGG 등)을 불러와서, 우리 문제(예: MNIST, CIFAR-10 등)에 맞게 마지막 레이어를 수정하는 방법을 학습합니다.

# 파인튜닝 전략:

모델의 일부 레이어를 고정(Frozen)하고, 마지막 몇 개 레이어만 학습하도록 설정하는 방법, 그리고 전체 네트워크를 미세 조정하는 방법을 배웁니다.

In [11]:
#Pre-trained ResNet18을 CIFAR-10에 맞게 파인튜닝
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader as loader
import matplotlib.pyplot as plt

transform_train=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023, 0.1994, 0.2010))
])

transform_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_data=datasets.CIFAR10(root="./data",train=True, download=True,transform=transform_train)
test_data=datasets.CIFAR10(root="./data",train=True,download=True, transform=transform_test)

train_loader=loader(train_data,batch_size=64,shuffle=True)
test_loader=loader(test_data, batch_size=1000, shuffle=True)

model=models.resnet18(pretrained=True)# 2. Pre-trained 모델 불러오기

# 3. Transfer Learning 적용: 마지막 FC 레이어 수정 (CIFAR-10은 10개 클래스)
num_ftrs=model.fc.in_features# 기존 fc layer의 입력 차원
model.fc=nn.Linear(num_ftrs,10)# CIFAR-10에 맞게 출력 클래스 수 수정

# 4. 손실 함수 및 옵티마이저 설정
criter=nn.CrossEntropyLoss()
optimy=optim.Adam(model.parameters(),lr=0.001)



100%|██████████| 170498071/170498071 [02:13<00:00, 1274319.10it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


C:\Users\JH\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\JH\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\JH/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:08<00:00, 5.42MB/s]


In [25]:
def train_mode(model, loader, criter, optimy, num_epoch=10):
    model.train()
    history=[]
    for epoch in range(num_epoch):
        total=0.0
        for data, target in loader:
            optimy.zero_grad()
            out=model(data)
            loss=criter(out,target)
            loss.backward()
            optimy.step()
            total+=loss.item()
        avg_loss=total/len(loader)
        history.append(avg_loss)
        print(f"Epoch {epoch+1}/{num_epoch}, Training Loss: {avg_loss:.4f}")
        return history

In [27]:
def eval_mode(model, loader):
    model.eval()
    correct=0.0
    total=0
    with torch.no_grad():
        for data, target in loader:
            out=model(data)
            pred=out.argmax(dim=1)
            correct+=pred.eq(target).sum().items()
            total+=data.size(0)
    acc=100.0*correct/total
    return acc

In [31]:
num_epoch=10
train=train_mode(model, train_loader, criter, optimy, num_epoch)
test_acc=eval_mode(model, test_loader)
print(f"Test Accuracy: {test_acc:.2f}%")


KeyboardInterrupt: 

In [ ]:
plt.plot(range(1,num_epoch+1),train,marker='o')

# 허깅페이스 ver

In [ ]:


# 필요한 라이브러리 불러오기
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# 1. 데이터셋 불러오기: 여기서는 GLUE의 MRPC 작업을 사용합니다.
dataset = load_dataset("glue", "mrpc")

# 2. 토크나이저 로드: 사전 학습된 BERT 토크나이저 사용
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 3. 데이터셋 토큰화 함수 정의
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"],
                     truncation=True, padding="max_length", max_length=128)

# 4. 데이터셋 토큰화 및 포맷 설정
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# 5. 사전 학습된 모델 불러오기 및 출력 레이어 수정 (num_labels 설정)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 6. 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# 7. Trainer 객체 생성 (모델, 학습 인자, 데이터셋 포함)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# 8. 모델 파인튜닝: Trainer를 이용해 학습 실행
trainer.train()


In [4]:
import torch
from torch.utils.data import DataLoader

# DataLoader 생성: 배치 크기 등 설정
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

# 옵티마이저 설정 (Fine-tuning 시 보통 낮은 학습률 사용)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 학습 루프
num_epochs = 3
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(input_ids=batch["input_ids"],
                        attention_mask=batch["attention_mask"],
                        labels=batch["labels"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


#전통적인 핛급

NameError: name 'train_dataset' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 1. 데이터 전처리 및 로딩 (CIFAR-10 예시)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 데이터 증강: 좌우 반전
    transforms.RandomCrop(32, padding=4), # 데이터 증강: 랜덤 크롭
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 2. Pre-trained 모델 불러오기: ResNet18 예시
model = models.resnet18(pretrained=True)

# 3. Transfer Learning 적용: 마지막 FC 레이어 수정 (CIFAR-10은 10개 클래스)
num_ftrs = model.fc.in_features  # 기존 fc layer의 입력 차원
model.fc = nn.Linear(num_ftrs, 10)  # CIFAR-10에 맞게 출력 클래스 수 수정

# (옵션) 미세 조정을 위해, 앞의 일부 레이어를 고정할 수도 있습니다.
# for param in model.parameters():
#     param.requires_grad = False
# for param in model.fc.parameters():
#     param.requires_grad = True

# 4. 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 간단한 학습 루프
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    train_loss_history = []
    for epoch in range(num_epochs):
        total_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        train_loss_history.append(avg_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}")
    return train_loss_history

# 6. 평가 함수
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            preds = output.argmax(dim=1)
            correct += preds.eq(target).sum().item()
            total += data.size(0)
    accuracy = 100.0 * correct / total
    return accuracy

# 7. 모델 학습 및 평가
num_epochs = 5
train_loss_history = train_model(model, train_loader, criterion, optimizer, num_epochs)
test_accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

# 8. 결과 시각화 (Training Loss)
plt.plot(range(1, num_epochs+1), train_loss_history, marker='o')
plt.xlabel("Epoch")
plt.ylabel("Average Training Loss")
plt.title("Training Loss over Epochs")
plt.show()


Files already downloaded and verified
Files already downloaded and verified
